# Essential dynamics

In [1]:
import torch
import pickle
from torch.utils.data import DataLoader
from torch import nn
import matplotlib.pyplot as plt
from itertools import combinations

import numpy as np
import seaborn as sns

from typing import Iterable
from sklearn.decomposition import PCA
from scipy.ndimage import gaussian_filter1d

sns.set_style("white")

plt.rcParams["figure.dpi"] = 300

In [2]:
DEVICE = ("cuda" if torch.cuda.is_available() else "cpu")
models = torch.load(f"../model1.pt", map_location=DEVICE)


In [3]:
def plot_explained_variance(pca, title="Explained Variance", ax: plt.Axes = None):
    num_pca_components = pca.n_components
    if ax is None:
        fig, ax = plt.subplots(figsize=(15, 8))

    ax.bar(
        range(num_pca_components), pca.explained_variance_ratio_[:num_pca_components]
    )
    for i, ratio in enumerate(pca.explained_variance_ratio_[:num_pca_components]):
        ax.text(i, ratio, f"{ratio:.2f}", fontsize=12, ha="center", va="bottom")
    ax.set_title(title)
    ax.set_xlabel("PC")
    ax.set_ylabel("Explained Variance")

    ax.set_xticks(range(num_pca_components), range(1, num_pca_components + 1))

In [4]:
def get_output_for_model(model: nn.Module, dataloader: DataLoader):
    model.to(DEVICE)
    logits = []
    with torch.no_grad():
        for xs, ys in dataloader:
            xs, ys = xs.to(model.device), ys.to(model.device)
            outputs = model(xs, ys)
            logits.append(outputs)
    logits = torch.cat(logits).to("cpu")
    return logits


def get_output_for_models(models: Iterable[nn.Module], dataloader: DataLoader):
    outputs = torch.stack([get_output_for_model(model, dataloader) for model in models])
    return outputs.reshape(len(outputs), -1)


def get_pca_components(samples, n_components):
    pca = PCA(n_components=n_components)
    transformed_samples = pca.fit_transform(samples)
    return pca, transformed_samples


def get_form_figure(pca, transformed_samples, figsize=(20, 4)):
    num_pca_components = pca.n_components
    pca_combos = list(combinations(range(num_pca_components), 2))
    num_pca_combos = len(pca_combos)
    fig, axes = plt.subplots(1, num_pca_combos + 1, figsize=figsize)
    for ax, (first_pca_index, second_pca_index) in zip(axes, pca_combos):
        sns.scatterplot(
            x=transformed_samples[:, first_pca_index],
            y=transformed_samples[:, second_pca_index],
            ax=ax,
            alpha=0.5,
            color="gray",
            s=10,
            legend=False,
        )
        ax.plot(
            transformed_samples[:, first_pca_index],
            transformed_samples[:, second_pca_index],
        )
        ax.set_xlabel(f"PC {first_pca_index+1}")
        ax.set_ylabel(f"PC {second_pca_index+1}")
        ax.set_title(f"PC {second_pca_index+1} vs PC {first_pca_index+1}")
    plot_explained_variance(pca, ax=axes[-1])
    plt.tight_layout(rect=[0, 0, 1, 1])
    return fig


def int_linspace(start, stop, num, return_type="list"):
    """
    Returns a set of integers evenly spaced on a linear scale.
    """
    result = set(int(i) for i in np.linspace(start, stop, num))
    if return_type == "set":
        return result
    result = sorted(list(result))
    if return_type == "list":
        return list(result)
    elif return_type == "np":
        return np.array(result)
    else:
        raise ValueError(
            f"return_type must be either 'list' or 'set', got {return_type}"
        )


def get_nearest_step(steps, step):
    idx = np.argmin(np.abs(np.array(steps) - step))
    return steps[idx]

In [5]:
BATCH_SIZE = 1024
dataset = torch.load("ED_dataset.pt")
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)
outputs = get_output_for_models(models, dataloader)


In [20]:
def sigma_helper(z, sigma_early, sigma_late, START_LERP, END_LERP):
    if z < START_LERP:
        return sigma_early
    elif z > END_LERP:
        return sigma_late
    else:
        return (sigma_late - sigma_early) / (END_LERP - START_LERP) * (
            z - START_LERP
        ) + sigma_early


def osculating_circle(curve, t_index):
    # Handle edge cases
    if t_index == 0:
        t_index = 1
    if t_index == len(curve) - 1:
        t_index = len(curve) - 2

    # Central differences for first and second derivatives
    r_prime = (curve[t_index + 1] - curve[t_index - 1]) / 2
    r_double_prime = curve[t_index + 1] - 2 * curve[t_index] + curve[t_index - 1]

    # Append a zero for 3D cross product
    r_prime_3d = np.append(r_prime, [0])
    r_double_prime_3d = np.append(r_double_prime, [0])

    # Curvature calculation and normal vector direction
    cross_product = np.cross(r_prime_3d, r_double_prime_3d)
    curvature = np.linalg.norm(cross_product) / np.linalg.norm(r_prime) ** 3
    signed_curvature = np.sign(cross_product[2])  # Sign of z-component of cross product
    radius_of_curvature = 1 / (curvature + 1e-12)

    # Unit tangent vector
    tangent = r_prime / np.linalg.norm(r_prime)

    # Unit normal vector, direction depends on the sign of the curvature
    if signed_curvature >= 0:
        norm_perp = np.array(
            [-tangent[1], tangent[0]]
        )  # Rotate tangent by 90 degrees counter-clockwise
    else:
        norm_perp = np.array(
            [tangent[1], -tangent[0]]
        )  # Rotate tangent by 90 degrees clockwise

    # Center of the osculating circle
    center = curve[t_index] + radius_of_curvature * norm_perp

    return center, radius_of_curvature


def get_smoothed_pcs(
    samples, num_pca_components, sigma_early, sigma_late, START_LERP, END_LERP
):
    smoothed_pcs = []
    for pca_component_i in range(0, num_pca_components):
        print(f"Processing smoothing for PC{pca_component_i+1}")
        smoothed_pc = np.copy(samples[:, :1])
        for z in range(len(samples)):
            sigma = sigma_helper(z, sigma_early, sigma_late, START_LERP, END_LERP)
            smoothed_pc[z, 0] = gaussian_filter1d(samples[:, pca_component_i], sigma)[z]
        smoothed_pcs.append(smoothed_pc)
    return smoothed_pcs


def get_osculate_center_and_radius(osculating_data, osculate_skip):
    dcenters = {}
    radii = {}
    circles = []
    prev_center = None
    for i, (z, (center, radius)) in enumerate(osculating_data.items()):
        if i % osculate_skip == 0:
            radii[z] = radius
            dcenters[z] = 1000
            if prev_center:
                dcenters[z] = np.linalg.norm(center - prev_center)
            circles.append(
                plt.Circle(
                    center, radius, alpha=0.5, color="lightgray", lw=0.5, fill=False
                )
            )
            prev_center = center
    return dcenters, radii, circles


def plot_essential_dynamics_grid(
    steps,
    samples,
    transitions=None,
    colors=None,
    num_pca_components=3,
    plot_caustic=True,
    figsize=(20, 6),
    num_sharp_points=20,
    num_vertices=35,
    osculate_start=1,
    osculate_end_offset=0,
    osculate_skip=8,
    smoothing_early=10,
    smoothing_late=60,
    smoothing_boundary=200,
):
    CUTOFF_START = osculate_start
    CUTOFF_END = osculate_end_offset
    OSCULATE_SKIP = osculate_skip
    LATE_BOUNDARY = smoothing_boundary
    START_LERP = 0.2 * LATE_BOUNDARY
    END_LERP = LATE_BOUNDARY

    if transitions and colors and len(colors) != len(transitions):
        print("todo warning")
    if len(samples) - CUTOFF_END - CUTOFF_START < 10:
        print("todo warning, less than 10 data points for osculates")

    pca_combos = list(combinations(range(num_pca_components), 2))
    num_pca_combos = len(pca_combos)
    fig, axes = plt.subplots(1, num_pca_combos, figsize=figsize)

    print("Number of samples: " + str(len(samples)))
    # Make sure we have the smoothed data for each PC
    smoothed_pcs = get_smoothed_pcs(
        samples,
        num_pca_components,
        smoothing_early,
        smoothing_late,
        START_LERP,
        END_LERP,
    )

    # Make sure we have the osculating data for each pair of PCs
    for ax, (second_pca_index, first_pca_index) in zip(axes, pca_combos):

        # For each PC pair we first do some data processing, then plotting
        print(f"Processing PC{second_pca_index+1} vs PC{first_pca_index+1}")
        smoothed_pc_i = smoothed_pcs[first_pca_index]
        smoothed_pc_j = smoothed_pcs[second_pca_index]
        smoothed_pc_ij_samples = np.column_stack((smoothed_pc_i, smoothed_pc_j))  # TODO
        osculating_data = {
            z: osculating_circle(smoothed_pc_ij_samples, z)
            for z in range(CUTOFF_START, len(samples) - CUTOFF_END)
        }
        dcenters, radii, circles = get_osculate_center_and_radius(
            osculating_data, OSCULATE_SKIP
        )
        for circle in circles:
            ax.add_artist(circle)

        # Find high curvature points (smallest radius so sort lo->hi and take first n)
        if len(radii) > num_sharp_points:
            upper_bound = np.sort(list(radii.values()))[num_sharp_points]
        else:
            upper_bound = 0
        # Mark in red high curvature points
        for z, radius in radii.items():
            if radius < upper_bound:
                print(f"Sharp point [{z}] curvature {radius}")
                ax.scatter(*smoothed_pc_ij_samples[z, :], color="red")

        # Find caustic cusps
        if len(dcenters) > num_vertices:
            dcenter_bound = np.sort(list(dcenters.values()))[-num_vertices]
        else:
            dcenter_bound = 0.0
        for z, dcenter in dcenters.items():
            if (
                z > 2 * OSCULATE_SKIP
                and dcenter < dcenter_bound
                and dcenters[z - OSCULATE_SKIP] < dcenter_bound
            ):
                print(f"Vertex [{z}] rate of change of osculate center {dcenter}")
                ax.scatter(
                    smoothed_pc_ij_samples[z, 0],
                    smoothed_pc_ij_samples[z, 1],
                    color="gold",
                )

        # Plot un-smoothed points in the background
        ax.scatter(
            x=samples[CUTOFF_START:-CUTOFF_END, first_pca_index],
            y=samples[CUTOFF_START:-CUTOFF_END, second_pca_index],
            alpha=0.5,
            color="lightgray",
            s=10,
        )
        if transitions:
            for color, (start, end, _) in zip(colors, transitions):
                start_idx = steps.index(get_nearest_step(steps, start))
                end_idx = steps.index(get_nearest_step(steps, end)) + 1
                ax.plot(
                    smoothed_pc_ij_samples[start_idx:end_idx, 0],
                    smoothed_pc_ij_samples[start_idx:end_idx, 1],
                    color=color,
                    lw=2,
                )
        else:
            ax.plot(samples[:, first_pca_index], samples[:, second_pca_index])

        # Draw the evolute
        if plot_caustic:
            current_x_limits, current_y_limits = ax.get_xlim(), ax.get_ylim()
            for (center_0, center_1), _ in osculating_data.values():
                if (current_x_limits[0] < center_0 < current_x_limits[1]) and (
                    current_y_limits[0] < center_1 < current_y_limits[1]
                ):
                    ax.scatter(center_0, center_1, color="black", s=0.5)
        #  if marked_cusp_data: # TODO
        #         for marked_cusp_id in range(len(marked_cusp_data)):
        #             marked_cusp = marked_cusp_data[marked_cusp_id]["step"]
        #             axes[I].scatter(
        #                 smoothed_samples[marked_cusp, 0],
        #                 smoothed_samples[marked_cusp, 1],
        #                 color="green",
        #                 marker="x",
        #                 s=40,
        #             )
        #             center, radius = osculating_data[marked_cusp]
        #             axes[I].scatter(
        #                 [center[0].item()],
        #                 [center[1].item()],
        #                 color="green",
        #                 marker="x",
        #                 s=60,
        #             )

        # if show_vertex_influence:# TODO
        #     vertex_influence_start = marked_cusp_data[marked_cusp_id][
        #         "influence_start"
        #     ]
        #     vertex_influence_end = marked_cusp_data[marked_cusp_id][
        #         "influence_end"
        #     ]
        #     axes[I].scatter(
        #         smoothed_samples[vertex_influence_start, 0],
        #         smoothed_samples[vertex_influence_start, 1],
        #         color="blue",
        #         marker="x",
        #         s=40,
        #     )
        #     axes[I].scatter(
        #         smoothed_samples[vertex_influence_end, 0],
        #         smoothed_samples[vertex_influence_end, 1],
        #         color="blue",
        #         marker="x",
        #         s=40,
        #     )
        ax.set_xlabel(f"PC {first_pca_index+1}")
        ax.set_ylabel(f"PC {second_pca_index+1}")

    if transitions:  # TODO
        legend_ax = fig.add_axes(
            [0.1, -0.03, 0.95, 0.05]
        )  # Adjust these values as needed
        handles = [
            plt.Line2D([0], [0], color=colors[i], linestyle="-")
            for i in range(len(transitions))
        ]
        labels = [label for _, _, label in transitions]
        legend_ax.legend(handles, labels, loc="center", ncol=len(labels), frameon=False)
        legend_ax.axis("off")

    plt.tight_layout(rect=[0, 0, 1, 1])
    fig.set_facecolor("white")
    return fig

In [21]:
_all_outputs = outputs
new_colors = [
    (1, 0.4980392156862745, 0.05490196078431375),
    (0.8392156862745098, 0.15294117647058825, 0.1568627450980392),
    (19 / 255, 177 / 255, 235 / 255),
    (19 / 255 * 0.6, 177 / 255 * 0.6, 255 / 255),
    (0.29411764705882354, 0.29411764705882354, 0.29411764705882354),
]
ED_FILENAME = "ED-LR-seed1.pkl"
steps = int_linspace(0, 500_000, 5_000)
TRANSITIONS = [
    (0, 1000, "LR1"),
    (1000, 40_000, "LR2"),
    (40000, 126000, "LR3"),
    (126000, 320000, "LR4"),
    (320000, 500000, "LR5"),
]
CLOSEST_TRANSITIONS = [
    (get_nearest_step(steps, t0), get_nearest_step(steps, t1), label)
    for t0, t1, label in TRANSITIONS
]
with open(ED_FILENAME, "rb") as file:
    pca, samples = pickle.load(file)
# pca, transformed_samples = get_pca_components(_all_outputs, n_components=3)
print("Explained variance of PCA")
print(pca.explained_variance_ratio_)
# fig = get_form_figure(pca, samples, figsize=(20, 3))
# fig.suptitle("First 10k Steps")
# plt.tight_layout()
# plt.show()

fig = plot_essential_dynamics_grid(
    steps,
    samples,
    transitions=CLOSEST_TRANSITIONS,
    colors=new_colors,
    num_pca_components=3,
    plot_caustic=True,
    figsize=(8, 8 / 10 * 6),
    num_sharp_points=0,
    num_vertices=0,
    osculate_start=1,
    osculate_end_offset=3000,
    osculate_skip=8,
    smoothing_early=1,
    smoothing_late=60,
    smoothing_boundary=200,
)
plt.tight_layout()
plt.show()

Explained variance of PCA
[0.5630448  0.15191932 0.0681259  0.0370252 ]
Number of samples: 5000
Processing smoothing for PC1


Processing smoothing for PC2
Processing smoothing for PC3
